In [22]:
import pandas
import numpy

In [23]:
data=pandas.read_csv('/content/data.csv')

In [24]:
data.head(1)

,id,topic,subtopic,persona,opening_question,messages
0,451c070a-78de-4a13-bafa-d4bdd697af9e,Space Law,Space Law Basics,"a busy person, needing quick and efficient ans...",Basics of Space Law? Role of international tre...,"[{'role': 'user', 'content': 'Basics of Space ..."


In [25]:
data.drop('topic',axis=1,inplace=True)
data.drop('subtopic',axis=1,inplace=True)
data.drop('persona',axis=1,inplace=True)

In [26]:
data.head(3)

,id,opening_question,messages
0,451c070a-78de-4a13-bafa-d4bdd697af9e,Basics of Space Law? Role of international tre...,"[{'role': 'user', 'content': 'Basics of Space ..."
1,2571369a-53ac-4dba-af23-6bf0c85ce3af,Could you provide a detailed analysis of the 1...,"[{'role': 'user', 'content': 'Could you provid..."
2,d6455bcf-a808-4087-8b2c-2da820646bd5,What are the specific impacts of the 1998 ISS ...,"[{'role': 'user', 'content': 'What are the spe..."


In [27]:
from datasets import Dataset

In [28]:
dataset=Dataset.from_pandas(data)

In [29]:
import ast

def convert_to_list(example):
    example['messages'] = ast.literal_eval(example['messages'])
    return example

dataset = dataset.map(convert_to_list)

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

In [30]:
dataset

Dataset({
    features: ['id', 'opening_question', 'messages'],
    num_rows: 901
})

In [31]:
dataset['messages'][0]

[{'content': 'Basics of Space Law? Role of international treaties? Space debris legalities? Commercial space activities law? Extraterrestrial resource extraction governance?',
  'role': 'user'},
 {'content': '### Basics of Space Law\n\nSpace Law encompasses the body of legal norms, rules, and principles that are applicable to activities in outer space. Initially developed to manage the exploration and use of outer space by nation-states, it has evolved to address issues like satellite communications, space exploration, and commercial space activities.\n\n### Role of International Treaties\n\nInternational treaties play a foundational role in Space Law, establishing principles that guide space activities. Key treaties include:\n\n1. **Outer Space Treaty (1967)**: Forms the basis of international space law, declaring space free for exploration and use by all states in a manner that benefits all humanity. It prohibits the placement of nuclear weapons in space and claims of sovereignty.\n2

In [6]:
model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer=AutoTokenizer.from_pretrained(model_id)

In [33]:
tokenizer

LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [34]:
tokenizer.padding_side='left'
tokenizer.truncation_side='left'

In [35]:
def format_prompt(example):
  chat=example['messages']
  prompt=tokenizer.apply_chat_template(chat,tokenize=False)

  return {'text':prompt}

In [36]:
tokenizer.apply_chat_template(dataset[0]['messages'],tokenize=False)

'<|user|>\nBasics of Space Law? Role of international treaties? Space debris legalities? Commercial space activities law? Extraterrestrial resource extraction governance?</s>\n<|assistant|>\n### Basics of Space Law\n\nSpace Law encompasses the body of legal norms, rules, and principles that are applicable to activities in outer space. Initially developed to manage the exploration and use of outer space by nation-states, it has evolved to address issues like satellite communications, space exploration, and commercial space activities.\n\n### Role of International Treaties\n\nInternational treaties play a foundational role in Space Law, establishing principles that guide space activities. Key treaties include:\n\n1. **Outer Space Treaty (1967)**: Forms the basis of international space law, declaring space free for exploration and use by all states in a manner that benefits all humanity. It prohibits the placement of nuclear weapons in space and claims of sovereignty.\n2. **Rescue Agreeme

In [37]:
dataset=dataset.map(format_prompt)

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

In [38]:
dataset['text'][0]

'<|user|>\nBasics of Space Law? Role of international treaties? Space debris legalities? Commercial space activities law? Extraterrestrial resource extraction governance?</s>\n<|assistant|>\n### Basics of Space Law\n\nSpace Law encompasses the body of legal norms, rules, and principles that are applicable to activities in outer space. Initially developed to manage the exploration and use of outer space by nation-states, it has evolved to address issues like satellite communications, space exploration, and commercial space activities.\n\n### Role of International Treaties\n\nInternational treaties play a foundational role in Space Law, establishing principles that guide space activities. Key treaties include:\n\n1. **Outer Space Treaty (1967)**: Forms the basis of international space law, declaring space free for exploration and use by all states in a manner that benefits all humanity. It prohibits the placement of nuclear weapons in space and claims of sovereignty.\n2. **Rescue Agreeme

In [39]:
from peft import LoraConfig, get_peft_model

In [19]:
from transformers import pipeline

In [ ]:
pipe=pipeline('text-generation',model=model_id,device='cuda')

Device set to use cuda


In [ ]:
prompt='''<|User|>
what are Basics of Space Law
<|Assistant|>'''
pipe(prompt)

[{'generated_text': '<|User|>\nwhat are Basics of Space Law\n<|Assistant|>\nSpace Law, also known as space law or space lawyering, is the set of rules and principles governing the exploration, use, and exploration of outer space. The basic principles of space law include:\n\n1. Self-Regulation: The United Nations Space Agency (UNOSAT) is responsible for developing and implementing standards and guidelines for space activities.\n\n2. International Treaties: The United Nations Office for Outer Space Affairs (UNOOSA) manages and promotes international treaties and agreements related to space.\n\n3. International Space Station (ISS): The ISS is an international platform for cooperation in space exploration, research, and development.\n\n4. International Space University (ISU): The ISU provides education, training, and research opportunities for space professionals and academics around the world.\n\n5. International Astronautical Congress (IAC): The IAC is a leading event for space professi

In [23]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [1]:
from transformers import BitsAndBytesConfig

In [2]:
import torch

In [3]:
bnbconfig=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16'
)

In [4]:
from transformers import AutoModelForCausalLM

In [12]:
model=AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnbconfig,device_map='auto')

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), e

In [13]:
model.config.use_cashe=False
model.config.pretraining_tp=1

In [14]:
target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj']

In [15]:
from peft import LoraConfig,get_peft_model

In [16]:
lora=LoraConfig(
    lora_alpha=32,
    target_modules=target_modules,
    r=32,
    task_type='CAUSAL_LM',
    bias="none",
    lora_dropout=0.05

)

In [17]:
from peft import prepare_model_for_kbit_training

In [18]:
model=prepare_model_for_kbit_training(model)

In [19]:
modl=get_peft_model(model,lora)

In [20]:
modl

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [21]:
from transformers import TrainingArguments,Trainer

In [44]:
import torch

In [66]:
args = TrainingArguments(
    output_dir='Train',
    eval_strategy='no',
    save_strategy='epoch',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,

    learning_rate=2e-4,
    optim="paged_adamw_8bit",

    num_train_epochs=3,
    max_steps=-1,

    eval_steps=None,
    torch_compile=False,
    torch_compile_backend="inductor",
    fp16=True,
    bf16=False,

    report_to='none',
    logging_steps=100,
    save_total_limit=2,
)

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.


In [50]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [51]:
from trl import SFTTrainer

In [67]:
trainer=SFTTrainer(
    model=modl,
    train_dataset=dataset,
    peft_config=lora,
    args=args
    )

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.


Converting train dataset to ChatML:   0%|          | 0/901 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/901 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/901 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2062 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/901 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [68]:
trainer.train()

Step,Training Loss
100,1.035700
200,1.006600
300,1.002400
400,0.998300
500,0.842400
600,0.749600
700,0.814700
800,0.806700
900,0.813700
1000,0.593300


TrainOutput(global_step=1353, training_loss=0.7904815927577389, metrics={'train_runtime': 2192.862, 'train_samples_per_second': 1.233, 'train_steps_per_second': 0.617, 'total_flos': 1.6531595464802304e+16, 'train_loss': 0.7904815927577389})

In [69]:
trainer.save_model("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")


('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/chat_template.jinja',
 'fine-tuned-model/tokenizer.model',
 'fine-tuned-model/added_tokens.json',
 'fine-tuned-model/tokenizer.json')

In [71]:
from transformers import pipeline

In [73]:
model = AutoModelForCausalLM.from_pretrained("fine-tuned-model", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-model")

In [74]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [79]:
prompt = """<|User|>
             ###Could you provide a detailed analysis of the 1998 ISS agreement and its impact on international space law?
          ###<|Assistant|>"""
response = pipe(prompt, max_new_tokens=800, do_sample=True, temperature=0.7)[0]["generated_text"]
print(response)

<|User|>
             ###Could you provide a detailed analysis of the 1998 ISS agreement and its impact on international space law?
          ###<|Assistant|>
           The 1998 International Space Station (ISS) Agreement, formally known as the Agreement on the International Space Station (IISA), is a landmark agreement that established a framework for cooperation between the United States, Russia, Canada, and other countries to build, maintain, and operate a permanent human spaceflight station in low Earth orbit. This agreement has significantly impacted international space law by establishing a legal basis for the international cooperation and sharing of space resources, the peaceful use of outer space, and the protection of the ISS against unauthorized access or use.

           **Key Points:**

           1. **Uniform Global Law:** It establishes a global legal framework that applies to all ISS activities, including those carried out by private companies or countries. This is a si